<a href="https://colab.research.google.com/github/Feranie/Hierarchical-Classification-Project/blob/main/IRH_Sele%C3%A7%C3%A3o_de_atributos_com_heur%C3%ADstica_Hill_Climbing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importando bibliotecas necessárias
import pandas as pd  # Biblioteca para manipulação de dados em formato de tabela
import random  # Biblioteca para geração de números aleatórios
from collections import defaultdict  # Dicionário com valor padrão para chaves inexistentes
from typing import List, Set, Tuple  # Tipos para anotações de tipo

# Classe para leitura de arquivos ARFF (formato usado pelo software WEKA para mineração de dados)
class ArffReader:
    def __init__(self):
        self.relation = ""  # Armazena o nome da relação/conjunto de dados
        self.attributes = []  # Lista para armazenar os atributos e seus tipos
        self.data = []  # Lista para armazenar os dados/instâncias

    # Método para ler um arquivo ARFF
    def read_arff(self, input_file):
        reading_data = False  # Flag para identificar quando estamos na seção de dados
        try:
            with open(input_file, 'r', encoding='utf-8') as file:  # Abre o arquivo com codificação UTF-8
                for line in file:  # Itera sobre cada linha do arquivo
                    line = line.strip()  # Remove espaços em branco no início e fim da linha
                    if not line or line.startswith('%'):  # Ignora linhas vazias ou comentários
                        continue
                    if line.lower().startswith('@relation'):  # Identifica a linha que define o nome da relação
                        self.relation = line.split(' ', 1)[1]  # Extrai o nome da relação
                    elif line.lower().startswith('@attribute'):  # Identifica linhas que definem atributos
                        parts = line.split(' ', 2)  # Divide a linha em até 3 partes
                        if len(parts) < 3:  # Verifica se a definição do atributo está completa
                            raise ValueError(f"Definição d'attribut invalide: {line}")  # Levanta erro se definição estiver incompleta
                        attribute_name = parts[1]  # Nome do atributo
                        attribute_type = parts[2]  # Tipo do atributo
                        self.attributes.append((attribute_name, attribute_type))  # Adiciona atributo à lista
                    elif line.lower().startswith('@data'):  # Identifica início da seção de dados
                        reading_data = True  # Ativa a flag de leitura de dados
                    elif reading_data:  # Se estiver na seção de dados
                        if ',' in line:  # Verifica se a linha contém dados separados por vírgula
                            values = line.split(',')  # Divide os valores
                            if len(values) == len(self.attributes):  # Verifica se o número de valores corresponde ao número de atributos
                                self.data.append(values)  # Adiciona os valores aos dados
                            else:
                                print(f"Avertissement: Ligne ignorée en raison d'un nombre incorrect de valeurs: {line}")  # Avisa sobre linhas ignoradas
            # Imprime informações sobre os dados lidos
            print(f"Fichier lu avec succès. Total d'instances: {len(self.data)}")
            print(f"Total d'attributs: {len(self.attributes)}")
            print(f"Exemples de données: {self.data[:5]}")
        except Exception as e:  # Captura qualquer erro durante a leitura
            print(f"Erreur lors de la lecture du fichier: {str(e)}")
            raise  # Propaga o erro

# Função para salvar dados em formato ARFF
def save_to_arff(df, output_file_path, relation_name, attributes):
    # Filtra os atributos para manter apenas os presentes no DataFrame
    filtered_attributes = [(name, type_) for name, type_ in attributes if name in df.columns or name == 'class']

    with open(output_file_path, 'w', encoding='utf-8') as f:  # Abre arquivo para escrita
        f.write(f"@relation {relation_name}\n\n")  # Escreve o nome da relação
        for attr_name, attr_type in filtered_attributes:  # Itera sobre os atributos filtrados
            f.write(f"@attribute {attr_name} {attr_type}\n")  # Escreve definição de cada atributo
        f.write("\n@data\n")  # Escreve marcador de início da seção de dados
        for index, row in df.iterrows():  # Itera sobre as linhas do DataFrame
            f.write(','.join(map(str, row.values)) + '\n')  # Converte valores para string e une com vírgulas
    print(f"Fichier ARFF sauvegardé avec succès en {output_file_path}")  # Confirma salvamento

# Função para obter o número de níveis na classificação hierárquica
def get_number_of_levels(data: pd.DataFrame) -> int:
    try:
        # Calcula o número máximo de níveis dividindo os valores da coluna 'class' por pontos
        return max(len(str(c).split('.')) for c in data['class'])
    except Exception as e:
        print(f"Erreur lors du calcul du nombre de niveaux : {e}")
        return 0  # Retorna 0 em caso de erro

# Função para calcular a taxa de inconsistência em um nível específico
def calculate_level_inconsistency_rate(data: pd.DataFrame, attribute_subset: List[str], level: int) -> float:
    pattern_counts = defaultdict(lambda: defaultdict(int))  # Dicionário para contar padrões de atributos
    total_instances = len(data)  # Total de instâncias no conjunto de dados

    # Conta ocorrências de cada padrão de atributos e classe no nível específico
    for _, row in data.iterrows():  # Itera sobre cada linha do DataFrame
        pattern = tuple(row[attr] for attr in attribute_subset)  # Cria um padrão com os valores dos atributos selecionados
        class_parts = str(row['class']).split('.')  # Divide a classe em níveis
        if len(class_parts) >= level:  # Verifica se a classe tem o nível necessário
            class_label = '.'.join(class_parts[:level])  # Obtém a classe até o nível especificado
            pattern_counts[pattern][class_label] += 1  # Incrementa a contagem para este padrão e classe

    # Calcula inconsistências (quando um mesmo padrão tem diferentes classes)
    inconsistency_count = 0
    for pattern, class_counts in pattern_counts.items():  # Para cada padrão e suas contagens de classe
        total_pattern_count = sum(class_counts.values())  # Total de instâncias com este padrão
        max_class_count = max(class_counts.values())  # Contagem da classe mais frequente
        pattern_inconsistency = total_pattern_count - max_class_count  # Instâncias inconsistentes
        inconsistency_count += pattern_inconsistency  # Acumula inconsistências

    # Retorna a taxa de inconsistência (inconsistências / total de instâncias)
    return inconsistency_count / total_instances if total_instances > 0 else float('inf')

# Função para calcular os pesos para cada nível hierárquico
def calculate_weights(h: int) -> List[float]:
    # Calcula pesos para cada nível, dando mais importância aos níveis superiores
    return [(h - i + 1) * 2 / (h * (h + 1)) for i in range(1, h + 1)]

# Função para calcular o IRH (Inconsistency Rate Hierarchy) - métrica para avaliar subconjuntos de atributos
def calculate_irh(data: pd.DataFrame, attribute_subset: List[str]) -> float:
    if not attribute_subset:  # Se o subconjunto estiver vazio
        return float('inf')  # Retorna infinito (pior caso)

    num_levels = get_number_of_levels(data)  # Obtém número de níveis hierárquicos
    weights = calculate_weights(num_levels)  # Calcula pesos para cada nível

    ir_levels = []  # Lista para armazenar taxas de inconsistência por nível
    for level in range(1, num_levels + 1):  # Para cada nível
        ir = calculate_level_inconsistency_rate(data, attribute_subset, level)  # Calcula taxa de inconsistência
        ir_levels.append(ir)  # Adiciona à lista

    # Retorna a soma ponderada das taxas de inconsistência
    return sum(w * ir for w, ir in zip(weights, ir_levels))

# Função para obter vizinhos de um subconjunto (adicionar ou remover um atributo)
def get_neighbors(current_subset: Set[str], all_features: List[str]) -> List[Set[str]]:
    neighbors = []  # Lista para armazenar subconjuntos vizinhos

    # Adiciona um atributo
    for feature in all_features:  # Para cada atributo disponível
        if feature not in current_subset:  # Se não estiver no subconjunto atual
            neighbor = current_subset | {feature}  # Adiciona o atributo ao subconjunto
            neighbors.append(neighbor)  # Adiciona à lista de vizinhos

    # Remove um atributo
    for feature in current_subset:  # Para cada atributo no subconjunto atual
        neighbor = current_subset - {feature}  # Remove o atributo do subconjunto
        if neighbor:  # Apenas adiciona se não ficar vazio
            neighbors.append(neighbor)  # Adiciona à lista de vizinhos

    return neighbors  # Retorna todos os vizinhos

# Algoritmo de seleção de atributos por subida de encosta (hill climbing)
def hill_climbing_feature_selection(data: pd.DataFrame, max_iterations: int = 100) -> Tuple[Set[str], float]:
    all_features = [col for col in data.columns if col != 'class']  # Lista de todos os atributos exceto a classe

    current_subset = set()  # Inicia com conjunto vazio de atributos
    current_irh = calculate_irh(data, list(current_subset))  # Calcula IRH inicial

    iteration = 0  # Contador de iterações
    while iteration < max_iterations:  # Enquanto não atingir número máximo de iterações
        # Imprime informações da iteração atual
        print(f"\nIteration {iteration + 1}")
        print(f"Current subset: {current_subset}")
        print(f"Current IRH: {current_irh:.4f}")

        neighbors = get_neighbors(current_subset, all_features)  # Obtém vizinhos
        best_neighbor = None  # Melhor vizinho encontrado
        best_neighbor_irh = float('inf')  # IRH do melhor vizinho (inicialmente infinito)

        # Avalia cada vizinho
        for neighbor in neighbors:  # Para cada vizinho
            neighbor_irh = calculate_irh(data, list(neighbor))  # Calcula IRH do vizinho
            print(f"Evaluating subset {neighbor}: IRH = {neighbor_irh:.4f}")  # Imprime avaliação

            if neighbor_irh < best_neighbor_irh:  # Se encontrou vizinho melhor
                best_neighbor = neighbor  # Atualiza melhor vizinho
                best_neighbor_irh = neighbor_irh  # Atualiza IRH do melhor vizinho

        if best_neighbor_irh >= current_irh:  # Se nenhum vizinho é melhor que o atual
            print("\nLocal optimum reached!")  # Atingiu ótimo local
            break  # Encerra a busca

        # Atualiza para o melhor vizinho
        current_subset = best_neighbor
        current_irh = best_neighbor_irh
        iteration += 1  # Incrementa contador de iterações

    # Imprime resultado final
    print("\nAtributos selecionados apos the ranking :", current_subset)
    print(f"IRH final : {current_irh:.4f}")

    return current_subset, current_irh  # Retorna o melhor subconjunto e seu IRH

# Função principal
def main():
    input_file_path = '/content/GPCR-PrintsTRA6.arff'  # Caminho do arquivo de entrada
    output_file_path = '/content/GPCR-PrintsTRA6Optimized.arff'  # Caminho do arquivo de saída

    print("Carregamento de dados...")  # Informa início do carregamento
    reader = ArffReader()  # Cria instância do leitor ARFF
    try:
        reader.read_arff(input_file_path)  # Tenta ler o arquivo
    except Exception as e:  # Captura erros de leitura
        print(f"Erreur lors de la lecture du fichier : {e}")
        return  # Encerra execução em caso de erro

    # Converte dados lidos para DataFrame pandas
    data = pd.DataFrame(reader.data, columns=[attr[0] for attr in reader.attributes])
    print(f"Total d'instances: {len(data)}")
    print(f"Total d'attributs: {len(data.columns)}")

    # Executa algoritmo de seleção de atributos
    best_subset, final_irh = hill_climbing_feature_selection(data)

    # Cria DataFrame apenas com os atributos selecionados e a classe
    selected_columns = list(best_subset) + ['class']
    optimized_data = data[selected_columns]

    # Salva os dados otimizados em formato ARFF
    save_to_arff(optimized_data, output_file_path, reader.relation, reader.attributes)

# Ponto de entrada do programa
if __name__ == "__main__":
    main()  # Executa a função principal